In [1]:
import pandas as pd

In [3]:
matches = pd.read_csv("matches.csv", index_col=0)

In [4]:
matches.head()

,Date,Time,Day,Venue,Result,GF,GA,Opponent
Team,,,,,,,,
Aegean Hawks FC,2022-09-11,20:00,Sunday,Home,W,3,2,World Class Premier
Aegean Hawks FC,2022-09-21,21:00,Wednesday,Home,W,8,1,Borussia Dulles
Aegean Hawks FC,2022-09-25,18:00,Sunday,Home,W,2,1,Los Toros
Aegean Hawks FC,2022-10-02,20:00,Sunday,Away,W,7,0,Esperion
Aegean Hawks FC,2022-10-09,20:00,Sunday,Home,L,0,2,Tysons City FC


In [5]:
matches.shape

(299, 8)

In [6]:
matches.dtypes

Date        object
Time        object
Day         object
Venue       object
Result      object
GF           int64
GA           int64
Opponent    object
dtype: object

In [11]:
matches["Date"] = pd.to_datetime(matches["Date"])

In [43]:
matches

,Date,Time,Day,Venue,Result,GF,GA,Opponent,date,venue,opp,hour,day_code,Target
Team,,,,,,,,,,,,,,
Aegean Hawks FC,2022-09-11,20:00,Sunday,Home,W,3,2,World Class Premier,2022-09-11,1,17,20,6,1
Aegean Hawks FC,2022-09-21,21:00,Wednesday,Home,W,8,1,Borussia Dulles,2022-09-21,1,3,21,2,1
Aegean Hawks FC,2022-09-25,18:00,Sunday,Home,W,2,1,Los Toros,2022-09-25,1,13,18,6,1
Aegean Hawks FC,2022-10-02,20:00,Sunday,Away,W,7,0,Esperion,2022-10-02,0,6,20,6,1
Aegean Hawks FC,2022-10-09,20:00,Sunday,Home,L,0,2,Tysons City FC,2022-10-09,1,16,20,6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yinz United,2024-10-27,20:00,Sunday,Away,W,4,2,World Class Premier,2024-10-27,0,17,20,6,1
Yinz United,2024-11-03,20:00,Sunday,Home,W,4,0,North Starz,2024-11-03,1,14,20,6,1
Yinz United,2024-11-17,20:00,Sunday,Home,W,2,1,World Class Premier,2024-11-17,1,17,20,6,1


In [14]:
matches.dtypes

Date        datetime64[ns]
Time                object
Day                 object
Venue               object
Result              object
GF                   int64
GA                   int64
Opponent            object
date        datetime64[ns]
dtype: object

In [50]:
matches["venue_code"] = matches["Venue"].astype("category").cat.codes

In [51]:
matches["opp_code"] = matches["Opponent"].astype("category").cat.codes

In [52]:
matches["hour"] = matches["Time"].str.replace(":.+", "", regex=True).astype("int")

In [53]:
matches["day_code"] = matches["Date"].dt.dayofweek

In [54]:
matches["Target"] = (matches["Result"] == "W").astype("int")

In [55]:
from sklearn.ensemble import RandomForestClassifier

In [56]:
rf = RandomForestClassifier(n_estimators=50, min_samples_split=10, random_state=1)

In [67]:
train = matches[matches["date"] < '2023-01-01']

In [68]:
test = matches[matches["date"] > '2023-01-01']

In [69]:
predictors = ["venue_code", "opp_code", "hour", "day_code"]

In [70]:
rf.fit(train[predictors], train["Target"])

RandomForestClassifier(min_samples_split=10, n_estimators=50, random_state=1)

In [71]:
preds = rf.predict(test[predictors])

In [72]:
from sklearn.metrics import accuracy_score

In [73]:
acc = accuracy_score(test["Target"], preds)

In [74]:
acc

0.5276381909547738

In [77]:
combined = pd.DataFrame(dict(actual=test["Target"], prediction=preds))

In [78]:
pd.crosstab(index=combined["actual"], columns=combined["prediction"])

prediction,0,1
actual,,
0,72,43
1,51,33


In [79]:
from sklearn.metrics import precision_score

In [80]:
precision_score(test["Target"], preds)

0.4342105263157895

In [81]:
matches

,Date,Time,Day,Venue,Result,GF,GA,Opponent,date,venue,opp,hour,day_code,Target,venue_code,opp_code
Team,,,,,,,,,,,,,,,,
Aegean Hawks FC,2022-09-11,20:00,Sunday,Home,W,3,2,World Class Premier,2022-09-11,1,17,20,6,1,1,17
Aegean Hawks FC,2022-09-21,21:00,Wednesday,Home,W,8,1,Borussia Dulles,2022-09-21,1,3,21,2,1,1,3
Aegean Hawks FC,2022-09-25,18:00,Sunday,Home,W,2,1,Los Toros,2022-09-25,1,13,18,6,1,1,13
Aegean Hawks FC,2022-10-02,20:00,Sunday,Away,W,7,0,Esperion,2022-10-02,0,6,20,6,1,0,6
Aegean Hawks FC,2022-10-09,20:00,Sunday,Home,L,0,2,Tysons City FC,2022-10-09,1,16,20,6,0,1,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yinz United,2024-10-27,20:00,Sunday,Away,W,4,2,World Class Premier,2024-10-27,0,17,20,6,1,0,17
Yinz United,2024-11-03,20:00,Sunday,Home,W,4,0,North Starz,2024-11-03,1,14,20,6,1,1,14
Yinz United,2024-11-17,20:00,Sunday,Home,W,2,1,World Class Premier,2024-11-17,1,17,20,6,1,1,17
